<a href="https://colab.research.google.com/github/rosinys/Social-Media-Performance-Funnel-Analysis/blob/main/Social_Media_Advertisement_Performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **📱 Social Media Advertisement Performance**

### 📊 Campaign Data Manipulation and Cleaning  
*Updated: October 2025 | Version 1*

🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹🔹

## 📥 Load and Read the Data

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("alperenmyung/social-media-advertisement-performance")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'social-media-advertisement-performance' dataset.
Path to dataset files: /kaggle/input/social-media-advertisement-performance


In [ ]:
import os
import pandas as pd
# 'path' is returned by kagglehub
print("Path:", path)
print("Contents of path:", os.listdir(path))

Path: /kaggle/input/social-media-advertisement-performance
Contents of path: ['users.csv', 'ad_campaign_db.sqlite', 'ads.csv', 'campaigns.csv', 'ad_events.csv']


In [ ]:
df_users = pd.read_csv(os.path.join(path, "users.csv"))
df_ads = pd.read_csv(os.path.join(path, "ads.csv"))
df_campaigns = pd.read_csv(os.path.join(path, "campaigns.csv"))
df_ad_events = pd.read_csv(os.path.join(path, "ad_events.csv"))

## 🔍 Explore the Data

### 🔹 Explore Users Table

In [ ]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      10000 non-null  object
 1   user_gender  10000 non-null  object
 2   user_age     10000 non-null  int64 
 3   age_group    10000 non-null  object
 4   country      10000 non-null  object
 5   location     10000 non-null  object
 6   interests    10000 non-null  object
dtypes: int64(1), object(6)
memory usage: 547.0+ KB


In [ ]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      10000 non-null  object
 1   user_gender  10000 non-null  object
 2   user_age     10000 non-null  int64 
 3   age_group    10000 non-null  object
 4   country      10000 non-null  object
 5   location     10000 non-null  object
 6   interests    10000 non-null  object
dtypes: int64(1), object(6)
memory usage: 547.0+ KB


In [ ]:
duplicate_count = df_users['user_id'].duplicated().sum()
print(f"Total duplicate user_id rows: {duplicate_count}")


Total duplicate user_id rows: 50


In [ ]:
if duplicate_count > 0:
    duplicated_ids = df_users.loc[df_users['user_id'].duplicated(), 'user_id'].unique()
    print(f"\nDuplicated user_id values ({len(duplicated_ids)} unique IDs):")
    print(duplicated_ids)



Duplicated user_id values (50 unique IDs):
['bf4c0' '0ebd6' '5a2cd' '3fc94' '878b1' 'b5d0b' 'bc9c0' '9b960' '15f7b'
 '9e923' 'fa7b1' 'c8620' 'f5eeb' '6f233' '94f30' '47824' '6102b' 'ce3fc'
 '2af0c' '9ec3a' '7984e' '11bbd' '49c5c' '02ad5' '5a190' '324bf' '168cf'
 'e9669' 'e3123' '2cff5' '310a0' '97409' 'c4f8e' '953e8' '699a5' '7a9e9'
 'b7687' 'ed6be' '4f67d' '7332b' '877a7' 'f234b' '23c32' '8a3cb' '1dc9b'
 '61037' 'adc26' 'a74fe' '5bdf4' '0b8c2']


In [ ]:
# Find duplicated user_ids (any that appear more than once)
dup_ids = df_users['user_id'][df_users['user_id'].duplicated(keep=False)]

# Remove ALL rows where user_id appears more than once
df_users_clean = df_users[~df_users['user_id'].isin(dup_ids)]


In [ ]:
df_users_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9900 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      9900 non-null   object
 1   user_gender  9900 non-null   object
 2   user_age     9900 non-null   int64 
 3   age_group    9900 non-null   object
 4   country      9900 non-null   object
 5   location     9900 non-null   object
 6   interests    9900 non-null   object
dtypes: int64(1), object(6)
memory usage: 618.8+ KB


In [ ]:
# Unique values in gender
print("Unique values in 'user_gender':")
print(df_users_clean['user_gender'].unique())

# Unique values in country
print("\nUnique values in 'country':")
print(df_users_clean['country'].unique())


Unique values in 'user_gender':
['Female' 'Male' 'Other']

Unique values in 'country':
['United Kingdom' 'Germany' 'Australia' 'India' 'United States' 'Mexico'
 'Canada' 'Japan' 'Brazil' 'France']


#### Creation of df_user_interests

In [ ]:
# Peek at the interests column
df_users_clean['interests'].head(10)


,interests
0,"fitness, health"
1,"food, fitness, lifestyle"
2,"fashion, news"
3,"health, news, finance"
4,"health, photography, lifestyle"
5,fitness
6,"health, technology, news"
7,travel
8,fashion
9,"gaming, lifestyle"


In [ ]:
df_users_clean.loc[:, 'interests'] = (
    df_users_clean['interests']
    .astype(str)
    .str.lower()
    .str.replace(r"[\[\]']", "", regex=True)   # remove brackets or quotes if any
    .str.replace(r"[;|]", ",", regex=True)     # replace semicolons/pipes with commas
    .str.replace(r"\s*,\s*", ",", regex=True)  # clean up spaces around commas
)


In [ ]:
# Create exploded table: user_id - interest
df_user_interests = (
    df_users_clean
    .dropna(subset=['interests'])
    .assign(interest=df_users_clean['interests'].str.split(','))
    .explode('interest')
    .drop(columns=['interests'])
)

# Clean any leftover whitespace
df_user_interests['interest'] = df_user_interests['interest'].str.strip()

In [ ]:
# Preview
display(df_user_interests.head(10))

,user_id,user_gender,user_age,age_group,country,location,interest
0,a2474,Female,24,18-24,United Kingdom,New Mariomouth,fitness
0,a2474,Female,24,18-24,United Kingdom,New Mariomouth,health
1,141e5,Male,21,18-24,Germany,Danielsfort,food
1,141e5,Male,21,18-24,Germany,Danielsfort,fitness
1,141e5,Male,21,18-24,Germany,Danielsfort,lifestyle
2,34db0,Male,27,25-34,Australia,Vincentchester,fashion
2,34db0,Male,27,25-34,Australia,Vincentchester,news
3,20d08,Female,28,25-34,India,Lisaport,health
3,20d08,Female,28,25-34,India,Lisaport,news
3,20d08,Female,28,25-34,India,Lisaport,finance


In [ ]:
print(f"Unique users in exploded table: {df_user_interests['user_id'].nunique()}")
print(f"Total interest entries: {len(df_user_interests)}")

print("\nTop 20 most common interests:")
print(df_user_interests['interest'].value_counts().head(20))


Unique users in exploded table: 9900
Total interest entries: 19734

Top 20 most common interests:
interest
fitness        1566
technology     1552
art            1534
gaming         1533
lifestyle      1530
travel         1530
fashion        1526
health         1515
sports         1507
news           1496
food           1494
finance        1479
photography    1472
Name: count, dtype: int64


In [ ]:
df_users_clean = df_users_clean.drop(columns=['interests'])


In [ ]:
df_users_clean.head(5)

,user_id,user_gender,user_age,age_group,country,location
0,a2474,Female,24,18-24,United Kingdom,New Mariomouth
1,141e5,Male,21,18-24,Germany,Danielsfort
2,34db0,Male,27,25-34,Australia,Vincentchester
3,20d08,Female,28,25-34,India,Lisaport
4,9e830,Male,28,25-34,United States,Brownmouth


In [ ]:
# Keep only first and last columns
df_user_interests = df_user_interests[[df_user_interests.columns[0], df_user_interests.columns[-1]]]

In [ ]:
df_user_interests.head(4)

,user_id,interest
0,a2474,fitness
0,a2474,health
1,141e5,food
1,141e5,fitness


### 🔹 Explore Ads Table

In [ ]:
df_ads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ad_id             200 non-null    int64 
 1   campaign_id       200 non-null    int64 
 2   ad_platform       200 non-null    object
 3   ad_type           200 non-null    object
 4   target_gender     200 non-null    object
 5   target_age_group  200 non-null    object
 6   target_interests  200 non-null    object
dtypes: int64(2), object(5)
memory usage: 11.1+ KB


In [ ]:
df_ads.describe()

,ad_id,campaign_id
count,200.000000,200.000000
mean,100.500000,25.125000
std,57.879185,13.713485
min,1.000000,1.000000
25%,50.750000,13.000000
50%,100.500000,25.000000
75%,150.250000,37.000000
max,200.000000,50.000000


In [ ]:
# Check for duplicate ad_id
duplicate_ad_id_count = df_ads['ad_id'].duplicated().sum()
print(f"Duplicate ad_id rows: {duplicate_ad_id_count}")

Duplicate ad_id rows: 0


In [ ]:
# Unique ad platforms
print("Unique ad_platform values:")
print(df_ads['ad_platform'].unique())

# Unique ad types
print("\nUnique ad_type values:")
print(df_ads['ad_type'].unique())

print("Unique target_gender values:")
print(df_ads['target_gender'].unique())

print("Unique target_age_group values:")
print(df_ads['target_age_group'].unique())

Unique ad_platform values:
['Facebook' 'Instagram']

Unique ad_type values:
['Video' 'Stories' 'Carousel' 'Image']
Unique target_gender values:
['Female' 'All' 'Male']
Unique target_age_group values:
['35-44' '25-34' '18-24' 'All']


In [ ]:
display(df_ads['target_interests'].head(10))


,target_interests
0,"art, technology"
1,"travel, photography"
2,technology
3,news
4,news
5,"finance, health"
6,"fashion, gaming"
7,technology
8,"fitness, lifestyle"
9,art


In [ ]:
# Clean target_interests column
df_ads['target_interests'] = (
    df_ads['target_interests']
    .astype(str)
    .str.lower()
    .str.replace(r"[\[\]']", "", regex=True)   # remove brackets or quotes if any
    .str.replace(r"[;|]", ",", regex=True)     # replace semicolons/pipes with commas
    .str.replace(r"\s*,\s*", ",", regex=True)  # remove extra spaces around commas
)


In [ ]:
# Create exploded table: ad_id - interest
df_ad_interests = (
    df_ads
    .dropna(subset=['target_interests'])
    .assign(interest=df_ads['target_interests'].str.split(','))
    .explode('interest')
    .drop(columns=['target_interests'])
)

# Clean whitespace
df_ad_interests['interest'] = df_ad_interests['interest'].str.strip()

# Preview
display(df_ad_interests.head(10))

,ad_id,campaign_id,ad_platform,ad_type,target_gender,target_age_group,interest
0,1,28,Facebook,Video,Female,35-44,art
0,1,28,Facebook,Video,Female,35-44,technology
1,2,33,Facebook,Stories,All,25-34,travel
1,2,33,Facebook,Stories,All,25-34,photography
2,3,20,Instagram,Carousel,All,25-34,technology
3,4,28,Facebook,Stories,Female,25-34,news
4,5,24,Instagram,Image,Female,25-34,news
5,6,33,Facebook,Video,Male,25-34,finance
5,6,33,Facebook,Video,Male,25-34,health
6,7,9,Facebook,Image,Female,18-24,fashion


In [ ]:
# Keep only first and last columns
df_ad_interests = df_ad_interests[[df_ad_interests.columns[0], df_ad_interests.columns[-1]]]
df_ad_interests = df_ad_interests.reset_index(drop=True)

In [ ]:
df_ad_interests.rename(columns={'interest': 'target_interest'}, inplace=True)

In [ ]:
df_ad_interests.head()

,ad_id,target_interest
0,1,art
1,1,technology
2,2,travel
3,2,photography
4,3,technology


In [ ]:
df_ads.drop(columns=['target_interests'], inplace=True)

In [ ]:
df_ads.head(4)

,ad_id,campaign_id,ad_platform,ad_type,target_gender,target_age_group
0,1,28,Facebook,Video,Female,35-44
1,2,33,Facebook,Stories,All,25-34
2,3,20,Instagram,Carousel,All,25-34
3,4,28,Facebook,Stories,Female,25-34


### 🔹 Explore Campaigns Table

In [ ]:
df_campaigns.head(5)

,campaign_id,name,start_date,end_date,duration_days,total_budget
0,1,Campaign_1_Launch,2025-05-25,2025-07-23,59,24021.32
1,2,Campaign_2_Launch,2025-04-16,2025-07-07,82,79342.41
2,3,Campaign_3_Winter,2025-05-04,2025-06-29,56,14343.25
3,4,Campaign_4_Summer,2025-06-04,2025-08-08,65,45326.60
4,5,Campaign_5_Launch,2025-07-11,2025-08-28,48,68376.69


In [ ]:
df_campaigns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   campaign_id    50 non-null     int64  
 1   name           50 non-null     object 
 2   start_date     50 non-null     object 
 3   end_date       50 non-null     object 
 4   duration_days  50 non-null     int64  
 5   total_budget   50 non-null     float64
dtypes: float64(1), int64(2), object(3)
memory usage: 2.5+ KB


In [ ]:
duplicate_campaigns = df_campaigns['campaign_id'].duplicated().sum()
print(f"Duplicate campaigns rows: {duplicate_campaigns}")

duplicate_campaigns_names= df_campaigns['name'].duplicated().sum()
print(f"Duplicate name rows: {duplicate_campaigns_names}")


Duplicate campaigns rows: 0
Duplicate name rows: 0


In [ ]:
# Check for duplicates in the combination of campaign_id and name
duplicate_combinations = df_campaigns.duplicated(subset=['campaign_id', 'name'], keep=False)

# Count how many rows are duplicates
num_duplicates = duplicate_combinations.sum()
print(f"Number of rows with duplicate (campaign_id, name) combination: {num_duplicates}")

# Optionally, display the duplicated rows
if num_duplicates > 0:
    display(df_campaigns[duplicate_combinations].sort_values(['campaign_id', 'name']))


Number of rows with duplicate (campaign_id, name) combination: 0


In [ ]:
# Convert start_date and end_date to datetime
df_campaigns['start_date'] = pd.to_datetime(df_campaigns['start_date'], errors='coerce')
df_campaigns['end_date'] = pd.to_datetime(df_campaigns['end_date'], errors='coerce')


In [ ]:
df_campaigns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   campaign_id    50 non-null     int64         
 1   name           50 non-null     object        
 2   start_date     50 non-null     datetime64[ns]
 3   end_date       50 non-null     datetime64[ns]
 4   duration_days  50 non-null     int64         
 5   total_budget   50 non-null     float64       
dtypes: datetime64[ns](2), float64(1), int64(2), object(1)
memory usage: 2.5+ KB


In [ ]:
df_campaigns.head()

,campaign_id,name,start_date,end_date,duration_days,total_budget
0,1,Campaign_1_Launch,2025-05-25,2025-07-23,59,24021.32
1,2,Campaign_2_Launch,2025-04-16,2025-07-07,82,79342.41
2,3,Campaign_3_Winter,2025-05-04,2025-06-29,56,14343.25
3,4,Campaign_4_Summer,2025-06-04,2025-08-08,65,45326.60
4,5,Campaign_5_Launch,2025-07-11,2025-08-28,48,68376.69


In [ ]:
# Format as MM-DD-YYYY for display
df_campaigns['start_date_formatted'] = df_campaigns['start_date'].dt.strftime('%m-%d-%Y')
df_campaigns['end_date_formatted'] = df_campaigns['end_date'].dt.strftime('%m-%d-%Y')

# Preview
display(df_campaigns[['campaign_id', 'start_date_formatted', 'end_date_formatted']].head())


,campaign_id,start_date_formatted,end_date_formatted
0,1,05-25-2025,07-23-2025
1,2,04-16-2025,07-07-2025
2,3,05-04-2025,06-29-2025
3,4,06-04-2025,08-08-2025
4,5,07-11-2025,08-28-2025


In [ ]:
df_campaigns['campaigns_duration_days'] = (df_campaigns['end_date'] - df_campaigns['start_date']).dt.days

In [ ]:
df_campaigns.head(5)

,campaign_id,name,start_date,end_date,duration_days,total_budget,start_date_formatted,end_date_formatted,campaigns_duration_days
0,1,Campaign_1_Launch,2025-05-25,2025-07-23,59,24021.32,05-25-2025,07-23-2025,59
1,2,Campaign_2_Launch,2025-04-16,2025-07-07,82,79342.41,04-16-2025,07-07-2025,82
2,3,Campaign_3_Winter,2025-05-04,2025-06-29,56,14343.25,05-04-2025,06-29-2025,56
3,4,Campaign_4_Summer,2025-06-04,2025-08-08,65,45326.60,06-04-2025,08-08-2025,65
4,5,Campaign_5_Launch,2025-07-11,2025-08-28,48,68376.69,07-11-2025,08-28-2025,48


In [ ]:
# Compare the two columns
all_match = (df_campaigns['duration_days'] == df_campaigns['campaigns_duration_days']).all()

print(f"Do all calculated durations match the existing 'campaigns_duration_days'? {all_match}")

# Optional: see rows where they do NOT match
mismatch_rows = df_campaigns[df_campaigns['duration_days'] != df_campaigns['campaigns_duration_days']]
print(f"Number of mismatched rows: {len(mismatch_rows)}")
display(mismatch_rows)


Do all calculated durations match the existing 'campaigns_duration_days'? True
Number of mismatched rows: 0


,campaign_id,name,start_date,end_date,duration_days,total_budget,start_date_formatted,end_date_formatted,campaigns_duration_days


In [ ]:
# Drop the campaigns_duration_days column
df_campaigns = df_campaigns.drop(columns=['campaigns_duration_days'])

In [ ]:
# Extract year, month, day
df_campaigns['start_year'] = df_campaigns['start_date'].dt.year
df_campaigns['start_month'] = df_campaigns['start_date'].dt.month
df_campaigns['start_day'] = df_campaigns['start_date'].dt.day

# Optional: same for end_date
df_campaigns['end_year'] = df_campaigns['end_date'].dt.year
df_campaigns['end_month'] = df_campaigns['end_date'].dt.month
df_campaigns['end_day'] = df_campaigns['end_date'].dt.day

In [ ]:
# Create a column for the day of the week from start_date
# .dt.day_name() returns full day name like 'Monday', 'Tuesday', etc.
df_campaigns['start_weekday'] = df_campaigns['start_date'].dt.day_name()

# Optional: also for end_date
df_campaigns['end_weekday'] = df_campaigns['end_date'].dt.day_name()

# Preview
df_campaigns.head(5)


,campaign_id,name,start_date,end_date,duration_days,total_budget,start_date_formatted,end_date_formatted,start_year,start_month,start_day,end_year,end_month,end_day,start_weekday,end_weekday
0,1,Campaign_1_Launch,2025-05-25,2025-07-23,59,24021.32,05-25-2025,07-23-2025,2025,5,25,2025,7,23,Sunday,Wednesday
1,2,Campaign_2_Launch,2025-04-16,2025-07-07,82,79342.41,04-16-2025,07-07-2025,2025,4,16,2025,7,7,Wednesday,Monday
2,3,Campaign_3_Winter,2025-05-04,2025-06-29,56,14343.25,05-04-2025,06-29-2025,2025,5,4,2025,6,29,Sunday,Sunday
3,4,Campaign_4_Summer,2025-06-04,2025-08-08,65,45326.60,06-04-2025,08-08-2025,2025,6,4,2025,8,8,Wednesday,Friday
4,5,Campaign_5_Launch,2025-07-11,2025-08-28,48,68376.69,07-11-2025,08-28-2025,2025,7,11,2025,8,28,Friday,Thursday


###

### 🔹 Explore Events Table

In [ ]:
df_ad_events.info()
df_ad_events.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   event_id     400000 non-null  int64 
 1   ad_id        400000 non-null  int64 
 2   user_id      400000 non-null  object
 3   timestamp    400000 non-null  object
 4   day_of_week  400000 non-null  object
 5   time_of_day  400000 non-null  object
 6   event_type   400000 non-null  object
dtypes: int64(2), object(5)
memory usage: 21.4+ MB


,event_id,ad_id,user_id,timestamp,day_of_week,time_of_day,event_type
0,1,197,2359b,2025-07-26 00:19:56,Saturday,Night,Like
1,2,51,f9c67,2025-06-15 08:28:07,Sunday,Morning,Share
2,3,46,5b868,2025-06-27 00:40:02,Friday,Night,Impression
3,4,166,3d440,2025-06-05 19:20:45,Thursday,Evening,Impression
4,5,52,68f1a,2025-07-22 08:30:29,Tuesday,Morning,Impression


In [ ]:
df_ad_events['timestamp'] = pd.to_datetime(df_ad_events['timestamp'], errors='coerce')

In [ ]:
df_ad_events['date'] = df_ad_events['timestamp'].dt.date
df_ad_events['hour'] = df_ad_events['timestamp'].dt.hour
df_ad_events['weekday'] = df_ad_events['timestamp'].dt.day_name()
df_ad_events['is_weekend'] = df_ad_events['weekday'].isin(['Saturday', 'Sunday'])

In [ ]:
df_ad_events.head()

,event_id,ad_id,user_id,timestamp,day_of_week,time_of_day,event_type,date,hour,weekday,is_weekend
0,1,197,2359b,2025-07-26 00:19:56,Saturday,Night,Like,2025-07-26,0,Saturday,True
1,2,51,f9c67,2025-06-15 08:28:07,Sunday,Morning,Share,2025-06-15,8,Sunday,True
2,3,46,5b868,2025-06-27 00:40:02,Friday,Night,Impression,2025-06-27,0,Friday,False
3,4,166,3d440,2025-06-05 19:20:45,Thursday,Evening,Impression,2025-06-05,19,Thursday,False
4,5,52,68f1a,2025-07-22 08:30:29,Tuesday,Morning,Impression,2025-07-22,8,Tuesday,False


In [ ]:
duplicate_events = df_ad_events['event_id'].duplicated().sum()
print(f"Duplicate event_id rows: {duplicate_events}")

Duplicate event_id rows: 0


In [ ]:
print("Unique event_type:", df_ad_events['event_type'].unique())
print("Unique day_of_week:", df_ad_events['day_of_week'].unique())
print("Unique time_of_day:", df_ad_events['time_of_day'].unique())


Unique event_type: ['Like' 'Share' 'Impression' 'Purchase' 'Click' 'Comment']
Unique day_of_week: ['Saturday' 'Sunday' 'Friday' 'Thursday' 'Tuesday' 'Monday' 'Wednesday']
Unique time_of_day: ['Night' 'Morning' 'Evening' 'Afternoon']


In [ ]:
day_time = ['Morning', 'Afternoon', 'Evening', 'Night']

# Loop through each category and show unique hours
for x in day_time:
    subset = df_ad_events[df_ad_events['time_of_day'] == x]
    unique_hours = subset['hour'].unique()
    print(f"{x} - Unique hours: {sorted(unique_hours)}")

Morning - Unique hours: [np.int32(6), np.int32(7), np.int32(8), np.int32(9), np.int32(10), np.int32(11)]
Afternoon - Unique hours: [np.int32(12), np.int32(13), np.int32(14), np.int32(15), np.int32(16), np.int32(17)]
Evening - Unique hours: [np.int32(18), np.int32(19), np.int32(20), np.int32(21), np.int32(22), np.int32(23)]
Night - Unique hours: [np.int32(0), np.int32(1), np.int32(2), np.int32(3), np.int32(4), np.int32(5)]


In [ ]:
# Vectorized update: set time_of_day to 'night' where hour is 22 or 23
df_ad_events.loc[df_ad_events['hour'].isin([22, 23]), 'time_of_day'] = 'Night'

In [ ]:
missing_ads = df_ad_events[~df_ad_events['ad_id'].isin(df_ads['ad_id'])]
missing_users = df_ad_events[~df_ad_events['user_id'].isin(df_users_clean['user_id'])]
print(len(missing_ads), len(missing_users))


0 3967


In [ ]:
# Keep only events where the user exists in df_users_clean
df_ad_events = df_ad_events[df_ad_events['user_id'].isin(df_users_clean['user_id'])].copy()

# Verify
print("Remaining events:", len(df_ad_events))


Remaining events: 396033


## Download the Data

In [ ]:
from google.colab import files

# Save df_ad_events as CSV
df_ad_events.to_csv('ad_events.csv', index=False)
df_campaigns.to_csv('campaigns.csv', index = False)
df_ads.to_csv('ads.csv', index = False)
df_users_clean.to_csv('users.csv', index = False)
df_ad_interests.to_csv('ad_target.csv', index = False)
df_user_interests.to_csv('df_user_interests.csv', index=False, header=True)



# Download the file
#files.download('ad_events.csv')
#files.download('campaigns.csv')
#files.download('ads.csv')
#files.download('users.csv')
#files.download('ad_target.csv')
#files.download('user_interests.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>